### Baseline experiment

Experiment to compare the 4 Optimization algorithms before trying to improve sparse search

In [6]:
import sys  
sys.path.insert(0, '../')

import HPO
import pysgpp
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import KFold
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from numpy.random import seed
import random
import time
import os
import math
from matplotlib import cm
import matplotlib 
import matplotlib.patches as patches

import operator

from scipy.stats import norm

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

random.seed(1)
seed(2)
tf.random.set_seed(3)

def reset_seeds():
    np.random.seed(1)
    random.seed(2)
    tf.random.set_seed(3)

VERBOSE = 0

SPARSE_RESULT = []
SPARSE_RESULT_OPTIMIZED = []

SPARSE_COST = []
SPARSE_COST_OPTIMIZED = []

### Hyperparameter space definition

In [7]:
hyperparameterspace = {
    'x0': ["interval", -5, 10],
    'x1': ["interval", -5, 10],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste



### Experiment parameters

In [8]:


BUDGETS = [1 + i * 52 for i in range(19)]

In [9]:
def visualize(points, adaptivity, alternative):
    x_values = []
    y_values = []
    z_values = []

    for i in range(len(points)):    
        x_values.append(points[i].get_coordinates()[0])
        y_values.append(points[i].get_coordinates()[1])
        z_values.append(points[i].get_value())


    fig = plt.figure()
    ax = plt.axes()
    surface = plt.scatter(x_values, y_values, c=z_values, cmap="plasma")
    plt.scatter(points[0].get_coordinates()[0], points[0].get_coordinates()[1], c="white", marker="x")


    ax.set_xlim([-5, 10])
    ax.set_ylim([-5, 10])

    plt.colorbar(surface)
    plt.gca().set_aspect("equal")

    # plt.savefig("./Alternative_"+str(alternative)+"_adapt_"+str(adaptivity)+".pgf",bbox_inches='tight' )

    plt.show()

### Optimization

In [10]:

################## MODEL AND FUNCTION DEFINITION ####################

def evaluate_model(epochs, batch_size):
    #return epochs * batch_size
    ################### ROSENBROCK ###############
    sum = 0
    sum += (1-epochs)**2 + 100 * (batch_size - epochs**2) ** 2
    return sum

    # ################# RASTRIGIN #################
    sum = 0
    sum += epochs ** 2 - 10 * math.cos(2 * math.pi * epochs)
    sum += batch_size ** 2 - 10 * math.cos(2 * math.pi * batch_size)
    
    return len(hyperparameterspace) * 10 + sum

def function(coordinates):
    return evaluate_model(coordinates[0], coordinates[1])


BUDGET = 200
INIT_POINTS = 5

ADAPTIVITIES = [0.1 * i for i in range(11)]


ALT_1 = "{"
ALT_2 = "{"
ALT_3 = "{"


for adaptivity in ADAPTIVITIES:

    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=0)
    points = optimizer.fit()

    points.sort(key=operator.attrgetter('value'))

    ALT_1 += "(" + str(adaptivity) + "," + str(points[0].get_value()) + ")"


    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=1)
    points = optimizer.fit()

    points.sort(key=operator.attrgetter('value'))
    

    ALT_2 += "(" + str(adaptivity) + "," + str(points[0].get_value()) + ")"



    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=2)
    points = optimizer.fit()

    points.sort(key=operator.attrgetter('value'))


    ALT_3 += "(" + str(adaptivity) + "," + str(points[0].get_value()) + ")"


print(ALT_1+"}")
print(ALT_2+"}")
print(ALT_3+"}")



{(0.0,1.517121125222536)(0.1,1.517121125222536)(0.2,1.517121125222536)(0.30000000000000004,1.517121125222536)(0.4,1.517121125222536)(0.5,1.517121125222536)(0.6000000000000001,1.5171211252086616)(0.7000000000000001,0.02466439437702513)(0.8,0.33502340624824123)(0.9,2.61442204654008)(1.0,4.566409386159776)}
{(0.0,0.008060015566753589)(0.1,0.008060015566753589)(0.2,0.008060015566753589)(0.30000000000000004,0.0023118954614058484)(0.4,0.002311894614521982)(0.5,0.008182947611344513)(0.6000000000000001,0.05339618203112462)(0.7000000000000001,0.08011233075458785)(0.8,2.1785202665483165)(0.9,1.5532894112998432)(1.0,55.21325276318493)}
{(0.0,0.04143540730358583)(0.1,0.04143540730358583)(0.2,0.04143540730358583)(0.30000000000000004,0.04143540730358583)(0.4,0.04143540730358583)(0.5,0.04143540730358583)(0.6000000000000001,0.1143331623160982)(0.7000000000000001,0.44676242767744595)(0.8,0.4676312056273927)(0.9,1.3722867702638328)(1.0,7.035490443472938)}
